In [ ]:
pip install selenium

In [ ]:
pip install selenium-stealth

In [ ]:
pip install requests


In [ ]:
pip install pandas

In [ ]:
pip install openpyxl

In [53]:
pip install 2captcha-python

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Batat\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [1]:
import pandas as pd 
import time
import random
from selenium import webdriver
from undetected_chromedriver import Chrome
from selenium_stealth import stealth
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from twocaptcha import TwoCaptcha

In [15]:
API_KEY = "22fcaab83c2393cd133542b3b8e75838"

url = "https://ww1.receita.fazenda.df.gov.br/DecVisualizador/Nfce/Captcha?Chave=53240617457404002589653180000642659019629963"

siteKey = "0x4AAAAAAAaOvnbOLak1uio1"


solver = TwoCaptcha(API_KEY)

try:
    result = solver.turnstile(sitekey=siteKey, url=url)
    print("CAPTCHA Solved:", result['code'])
except Exception as e:
    print("Error solving CAPTCHA:", str(e))
    result = None

if result:

    options = webdriver.ChromeOptions()
    service = Service()
    driver = webdriver.Chrome(options=options)

    driver.get(url)

    time.sleep(5)
    
    driver.execute_script(f'''
        var elements = document.querySelectorAll("[name='g-recaptcha-response']");
        for (var i = 0; i < elements.length; i++) {{
            elements[i].value = "{result['code']}";
        }}
    ''')
    
    time.sleep(2)

    button = driver.find_element(By.CLASS_NAME, "btn")
    button.get_attribute("value")

    button.click()

    # Pegando a lista com os produtos
    itens_str = driver.find_element(By.XPATH, "//ul").text

    itens_lista = itens_str.split('\n')

    itens_lista = list(filter(None, itens_lista))

# Convertendo a lista em um DataFrame do Pandas
    df_all_itens = pd.DataFrame(itens_lista, columns=['Itens'])

# Pegando apenas os registros que tem "Cod: blabla" (indica que é um produto)
    df_just_itens_name = df_all_itens[df_all_itens['Itens'].str.contains(r'\(Cód: \w+\)')].reset_index(drop=True)

# Limpeza para remover números e medidas dos nomes
    df_just_itens_name['Itens'] = df_just_itens_name['Itens'].str.replace(r' \(Cód: S\d+\)', '', regex=True)
    df_just_itens_name['Itens'] = df_just_itens_name['Itens'].str.replace(r'\d+[Xx]\d+\w*|\d+\w*|\d+', '', regex=True)
    df_just_itens_name['Itens'] = df_just_itens_name['Itens'].str.strip()

# Pegando a primeira palavra de cada registro
    df_just_itens_name['Primeira Palavra'] = df_just_itens_name['Itens'].str.split().str[0]

# Gerando um excel
    df_just_itens_name.to_excel("nota_fiscal.xlsx", index=False)


CAPTCHA Solved: 0.95WfoBOUuwBEcIAR6Rwk3u-5PX73x9R3TQQlQuEIWHy5_eZSQ5mmJrvyaF5a2h2C2OWa-VTUgWlyxhvJQlbubXvRH0IR115jc_f1o38o4at9_guFVFJfQfCo6PrINtmps4HZFZnHP8aH4C_THOSYkZhcUBT9XD4SFY8STw-TEvvsSXHRzbg_gV-O9IkY79ztEK7Z-d9Efbzerg7EaLzniFLC2UfpAgqRdQnb4iXaPhMI3rBgoiXZblpcGtqiZOT7oOnmVllOzd1hrIg3-zxZLDe3tqfY-bBZPpi9UQCwK4imAn3L8yX5HXuiUf_eBtjgvI_N4guTccY-NKnbzYsgLE4zQGJndNN_GH0f_s5Jv1ICSGiLb-Ulamyh78zwQdhjHCl5q9NzO0wCSsxa8xR-gnoTwQNaMi4rQDftcV-vmwgFOnoccdZghkjlSCpzROItTUUAwRqMA-KEVqklyxmjoA.QSnMVMwCg09U5gGYd43tTQ.88f02a352b5df58885305d84823121716a974eb5b233a8320e8cd8d3f25f5f66
